### IoU（交并比）

作用：用来衡量由边界框回归算法得到的预测目标框和真实目标框之间的重合程度。


In [5]:
import numpy as np
#np.maximum()依次比较两个array的数，选择max的
a=np.maximum([4,2],[3,4])
print(a)


[4 4]


In [8]:
import numpy as np
#np.clip(x,min,max),clip是夹子的意思,将x中的值夹到min与max之间
b=np.clip([0.3,-0.3,3],0,1)
print(b)

[0.3 0.  1. ]


In [2]:
import numpy as np

def IoU(vertice1,vertice2):
    """
    计算两个矩形区域的交并比函数，verticle格式为[xmin,ymin,xmax,ymax]
    xmin即为矩形框中xmin的那个
    返回两个矩形区域的交并比
    """
    #计算区域交集的左上与右下坐标(个人认为这个是左下与右上的坐标)
    lu=np.maximum(verticel[0:2],vertice2[0:2])
    rd=np.minimum(verticee1[2:],vertice2[2:])
    
    
    #得到交集区域的长宽
    intersection=np.maximum(0.0,rd-lu)
    #计算区域交集的面积
    inter_square=intersection[0]*intersection[1]
    
    #计算区域并集的面积
    square1=(vertice1[2]-vertice1[0])*(vertice1[3]-vertice1[1])
    square2=(vertice2[2]-vertice2[0])*(vertice2[3]-vertice2[1])
    union_square=np.maximumximum(square1+square2-inter_square,1e-10)
    
    #返回IoU
    return np.clip(inter_squarent/union_square,0.0,1.0)


### 非极大抑制算法（NMS）
作用：去除同一目标产生的大量重复区域

步骤：将所有的框按分类分数从高到低进行排名；选择得分最高的框，与其他框进行IoU的计算，若IoU大于一定阈值则将那个框删掉。一直循环遍历直到分数数组为空

In [11]:
import numpy as np
#argsort(),将数组中的元素按从小到大进行排序，输出排序后的index
x=np.array([1,2,3,4,5])
y=x.argsort()
print(y)

[0 1 2 3 4]


In [14]:
import numpy as np
#np.where(condition),返回index
#np.where(condition,x,y)满足条件返回x,不满足条件返回y
a=np.array([2,4,6,7,8,10])
b=np.where(a>5)
print(b)
c=np.where(a>5,1,-1)
print(c)

(array([2, 3, 4, 5], dtype=int64),)
[-1 -1  1  1  1  1]


In [ ]:
import numpy as np

def py_cpu_nms(dets,thresh):
    """
    这是NMS去除重复目标框的函数
    :param dets：目标框“数组”，目标框的格式为：[xmin,ymin,xmax,ymax,score]
    :param thresh: 阈值
    :return: 不重复的目标框数组在元目标框数组中的下标
    """
    vertices=dets[:,0:4] #所有目标框，元目标框数组
    scores=dets[:,4] #bbox打的分
    
    #打分排序，从大到小
    #[::-1]是倒序排序
    order= scores.argsort()[::-1]
    #keep为最后保留的边框
    keep=[]
    while order.size>0:
        #order[0]为当前分数数组中最大的那个，肯定要保留
        i=order[0]
        keep.append(i)
        #计算窗口i与其他所有窗口的交叠部分的面积
        ious=np.array([IoU(vertices[i],vertices[j])for j in order[1:]])
        #inds为所有与窗口i的iou值小于threshold值的窗口的index
        #保留那些小于阈值的候选框
        inds=np.where(ious<=thresh)[0]#为什么要加个[0]?
        #order里面只保留那些与窗口i的IoU小于threshold的那些窗口
        order=order[inds+1]#为啥要[inds+1]?
    return keep